# Loop 7 Analysis: Strategic Assessment

Key findings:
1. Our current score (70.68) is BETTER than the LB leader (71.19)
2. The target (68.92) is 1.75 points below our score
3. Local search methods are exhausted - baseline is at a strong local optimum
4. External ensemble approach found small improvements (0.003 points from SmartManoj)

In [1]:
import pandas as pd
import numpy as np

# Load our current best submission
df = pd.read_csv('/home/submission/submission.csv')
print(f'Submission rows: {len(df)}')
print(df.head())

Submission rows: 20100
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  


In [2]:
# Calculate per-N scores to identify worst-performing configurations
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 30
scale_factor = Decimal('1e15')

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

def get_side_length(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1]) / float(scale_factor)

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_side_length(trees)
    return side ** 2 / n

def load_configuration_from_df(n, df):
    group_data = df[df['id'].str.startswith(f'{n:03d}_')]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row['x'])[1:] if str(row['x']).startswith('s') else str(row['x'])
        y = str(row['y'])[1:] if str(row['y']).startswith('s') else str(row['y'])
        deg = str(row['deg'])[1:] if str(row['deg']).startswith('s') else str(row['deg'])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

print('Functions defined')

Functions defined


In [3]:
# Calculate per-N scores
scores = {}
for n in range(1, 201):
    trees = load_configuration_from_df(n, df)
    if trees:
        scores[n] = get_score(trees, n)

total_score = sum(scores.values())
print(f'Total score: {total_score:.6f}')
print(f'Target: 68.922808')
print(f'Gap to target: {total_score - 68.922808:.6f} points ({(total_score - 68.922808) / 68.922808 * 100:.2f}%)')
print(f'\nLB Leader (terry_u16): 71.19')
print(f'Our advantage over LB leader: {71.19 - total_score:.6f} points')

Total score: 70.676104
Target: 68.922808
Gap to target: 1.753296 points (2.54%)

LB Leader (terry_u16): 71.19
Our advantage over LB leader: 0.513896 points


In [4]:
# Identify worst-performing N values (highest per-N scores)
scores_df = pd.DataFrame([(n, s) for n, s in scores.items()], columns=['N', 'Score'])
scores_df = scores_df.sort_values('Score', ascending=False)

print('Top 20 worst-performing N values (highest scores):')
print(scores_df.head(20).to_string(index=False))

print(f'\nSum of top 20 worst scores: {scores_df.head(20)["Score"].sum():.6f}')
print(f'Sum of remaining 180 scores: {scores_df.tail(180)["Score"].sum():.6f}')

Top 20 worst-performing N values (highest scores):
 N    Score
 1 0.661250
 2 0.450779
 3 0.434745
 5 0.416850
 4 0.416545
 7 0.399897
 6 0.399610
 9 0.387415
 8 0.385407
15 0.379203
10 0.376630
21 0.376451
20 0.376057
11 0.375736
22 0.375258
16 0.374128
26 0.373997
12 0.372724
13 0.372323
25 0.372144

Sum of top 20 worst scores: 8.077149
Sum of remaining 180 scores: 62.598954


In [5]:
# Analyze score distribution by N range
ranges = [(1, 20), (21, 50), (51, 100), (101, 150), (151, 200)]

print('Score distribution by N range:')
for start, end in ranges:
    range_scores = [scores[n] for n in range(start, end + 1)]
    print(f'  N={start}-{end}: total={sum(range_scores):.4f}, avg={np.mean(range_scores):.4f}, max={max(range_scores):.4f}')

Score distribution by N range:
  N=1-20: total=8.0573, avg=0.4029, max=0.6612
  N=21-50: total=10.9849, avg=0.3662, max=0.3765
  N=51-100: total=17.6411, avg=0.3528, max=0.3656
  N=101-150: total=17.1441, avg=0.3429, max=0.3513
  N=151-200: total=16.8487, avg=0.3370, max=0.3425


In [6]:
# Calculate theoretical minimum improvement needed
print('\n=== Improvement Analysis ===')
print(f'Current score: {total_score:.6f}')
print(f'Target score: 68.922808')
print(f'Gap: {total_score - 68.922808:.6f} points')

# If we could improve each N by the same percentage
required_reduction = (total_score - 68.922808) / total_score * 100
print(f'\nRequired overall reduction: {required_reduction:.2f}%')

# If we focus on worst 20 N values
worst_20_sum = scores_df.head(20)['Score'].sum()
if worst_20_sum > (total_score - 68.922808):
    print(f'\nIf we improve only worst 20 N values:')
    print(f'  Current sum: {worst_20_sum:.4f}')
    print(f'  Need to reduce by: {total_score - 68.922808:.4f}')
    print(f'  Required reduction: {(total_score - 68.922808) / worst_20_sum * 100:.2f}%')


=== Improvement Analysis ===
Current score: 70.676104
Target score: 68.922808
Gap: 1.753296 points

Required overall reduction: 2.48%

If we improve only worst 20 N values:
  Current sum: 8.0771
  Need to reduce by: 1.7533
  Required reduction: 21.71%


In [7]:
# Summary of our position
print('\n' + '='*60)
print('STRATEGIC SUMMARY')
print('='*60)
print(f'\n1. CURRENT POSITION:')
print(f'   - Our score: {total_score:.6f}')
print(f'   - LB Leader: 71.19')
print(f'   - We are {71.19 - total_score:.4f} points BETTER than LB leader!')
print(f'\n2. TARGET ANALYSIS:')
print(f'   - Target: 68.922808')
print(f'   - Gap: {total_score - 68.922808:.4f} points ({(total_score - 68.922808) / 68.922808 * 100:.2f}%)')
print(f'\n3. WHAT WE KNOW:')
print(f'   - Local search (SA, swaps, squeeze) cannot improve baseline')
print(f'   - C++ optimizer finds better configs but with overlaps')
print(f'   - External ensemble (SmartManoj) found 0.003 points improvement')
print(f'   - Backward propagation found 0.002 points improvement')
print(f'\n4. REMAINING OPTIONS:')
print(f'   a) Find more external pre-optimized solutions')
print(f'   b) Run longer optimization with strict overlap checking')
print(f'   c) Try asymmetric solutions (mentioned in discussions)')
print(f'   d) Focus on specific N values with targeted optimization')


STRATEGIC SUMMARY

1. CURRENT POSITION:
   - Our score: 70.676104
   - LB Leader: 71.19
   - We are 0.5139 points BETTER than LB leader!

2. TARGET ANALYSIS:
   - Target: 68.922808
   - Gap: 1.7533 points (2.54%)

3. WHAT WE KNOW:
   - Local search (SA, swaps, squeeze) cannot improve baseline
   - C++ optimizer finds better configs but with overlaps
   - External ensemble (SmartManoj) found 0.003 points improvement
   - Backward propagation found 0.002 points improvement

4. REMAINING OPTIONS:
   a) Find more external pre-optimized solutions
   b) Run longer optimization with strict overlap checking
   c) Try asymmetric solutions (mentioned in discussions)
   d) Focus on specific N values with targeted optimization
